In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [13]:
metadata = pd.read_csv('./metadata.csv')

In [14]:
str_kor = []

for idx, data in metadata.iterrows():
    str_data = str(data['한국어'])  # 한국어 컬럼 데이터 String 변환
    str_kor.append(str_data)

print(f"Total class : {metadata['한국어'].nunique()}")

Total class : 419


In [20]:
label_encoder = LabelEncoder()  # 정수 인코딩 - LabelEncoder 선언
encoded_label = label_encoder.fit_transform(str_kor)    # 정수 인코딩

processed_label = to_categorical(encoded_label, num_classes=419)    # categorical transform
print(f"Encoded_label : {processed_label}")
print(F"Shape : {processed_label.shape}")

Encoded_label : [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape : (2000, 419)


In [21]:
keypoints_data = np.load('./pose_array.npy')
print(f"Shape : {keypoints_data.shape}")

Shape : (2000, 210, 33, 3)


In [22]:
class embedding_dnn_model(tf.keras.Model):
    def __init__(self):
        super(embedding_dnn_model, self).__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim=527, output_dim=512, input_length=1)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2048, activation='relu')
        self.dense3 = tf.keras.layers.Dense(4096, activation='relu')
        self.output_dense = tf.keras.layers.Dense(210 * 33 * 3, activation='linear')
        self.reshape = tf.keras.layers.Reshape((210, 33, 3))

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.output_dense(x)
        return self.reshape(x)

model = embedding_dnn_model()
model.compile(
    optimizer='adam', 
    loss='mean_squared_error',
    metrics=['mse']
)

model.build(input_shape=(None, 1)) # model input shape 지정

c:\Users\NEULET\anaconda3\envs\venv1\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\NEULET\anaconda3\envs\venv1\lib\site-packages\keras\src\layers\layer.py:372: UserWarning: `build()` was called on layer 'embedding_dnn_model_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [10]:
history = model.fit(processed_label, keypoints_data, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 942ms/step - loss: 1.0927 - mse: 1.0927 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 923ms/step - loss: 0.1664 - mse: 0.1664 - val_loss: 0.0863 - val_mse: 0.0863
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 894ms/step - loss: 0.0795 - mse: 0.0795 - val_loss: 0.0761 - val_mse: 0.0761
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 893ms/step - loss: 0.0642 - mse: 0.0642 - val_loss: 0.0561 - val_mse: 0.0561
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 888ms/step - loss: 0.0583 - mse: 0.0583 - val_loss: 0.0650 - val_mse: 0.0650
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 889ms/step - loss: 0.0577 - mse: 0.0577 - val_loss: 0.0556 - val_mse: 0.0556
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 892ms/step - loss: 0.0586 - mse: 0.0586 - val_loss: 0.0552 - val_mse: 0.0552
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 888ms/step - loss: 0.0587 - mse: 0.0587 - val_loss: 0.0538 - val_mse: 0.0538
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 898ms/

In [35]:
input_string = ["바다"]
encoded_string = label_encoder.transform(input_string)
array = np.array(encoded_string).reshape(-1, 1)

predicted = model.predict(array)

print(f"Predicted_shape: {predicted.shape}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted_shape: (1, 210, 33, 3)
